Homework-02

In [2]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def uniform_generator(size):
    return np.random.uniform(-1, 1, size)

# Пример использования
sample_uniform = uniform_generator(10)  # Генерация 10 случайных чисел
print("Равномерное распределение:", sample_uniform)


Равномерное распределение: [ 0.00388224 -0.81160067  0.50891619  0.39196022  0.43734859  0.29280671
  0.280665   -0.3493284  -0.75861872  0.80727332]


In [9]:
def normal_generator(N):
    samples = [random.gauss(0, 1) for _ in range(N)]
    return samples

sample_normal = normal_generator(10)  # Генерация 10 случайных чисел
print("Нормальное распределение:", sample_normal)



Нормальное распределение: [0.8063804656562525, 0.9994100863381945, 1.1489530047483552, -0.06351532507994001, -0.8173185336395913, -1.1538918209635705, 0.09559730588667706, 0.4409551694118019, 0.24265147364279666, -1.9846973019067746]


In [12]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Параметры
sample_sizes = [10, 100, 1000]
num_samples = 10000
true_mean_uniform = 0
true_var_uniform = (1 - (-1))**2 / 12 
true_mean_normal = 1
true_var_normal = 0 

def jackknife_ci(sample, alpha=0.05):
    n = len(sample)
    mean = np.mean(sample)
    
    jackknife_means = [(n * mean - x) / (n - 1) for x in sample]
    jackknife_se = np.sqrt((n - 1) / n * np.sum((jackknife_means - mean)**2))
    
    z_score = stats.norm.ppf(1 - alpha/2)
    # границы доверительного интервала
    ci_lower = mean - z_score * jackknife_se
    ci_upper = mean + z_score * jackknife_se
    
    return ci_lower, ci_upper

def bootstrap_ci(sample, alpha=0.05):
    n = len(sample)
    bootstrap_samples = np.random.choice(sample, (num_samples, n), replace=True)
    bootstrap_means = np.mean(bootstrap_samples, axis=1)
    # границы доверительного интервала
    lower_bound = np.percentile(bootstrap_means, (alpha/2) * 100)
    upper_bound = np.percentile(bootstrap_means, (1 - alpha/2) * 100)
    
    return lower_bound, upper_bound

def simulate(generator, true_mean, true_var):
    results_jackknife = []
    results_bootstrap = []
    
    for size in sample_sizes:
        contained_jackknife = 0
        contained_bootstrap = 0
        
        for _ in range(num_samples):
            sample = generator(size)
            
            # Jackknife
            ci_jackknife = jackknife_ci(sample)
            if ci_jackknife[0] <= true_mean <= ci_jackknife[1]:
                contained_jackknife += 1
            
            # Bootstrap
            ci_bootstrap = bootstrap_ci(sample)
            if ci_bootstrap[0] <= true_mean <= ci_bootstrap[1]:
                contained_bootstrap += 1
        
        results_jackknife.append(contained_jackknife / num_samples)
        results_bootstrap.append(contained_bootstrap / num_samples)
    
    return results_jackknife, results_bootstrap

# Симуляция для равномерного распределения
jackknife_uniform, bootstrap_uniform = simulate(uniform_generator, true_mean_uniform, true_var_uniform)

# Симуляция для нормального распределения
jackknife_normal, bootstrap_normal = simulate(normal_generator, true_mean_normal, true_var_normal)

# Вывод результатов
print("Доля интервалов для равномерного распределения:")
for size, j, b in zip(sample_sizes, jackknife_uniform, bootstrap_uniform):
    print(f"Размер выборки {size}: Jackknife: {j:.4f}, Bootstrap: {b:.4f}")

print("\nДоля интервалов для нормального распределения:")
for size, j, b in zip(sample_sizes, jackknife_normal, bootstrap_normal):
    print(f"Размер выборки {size}: Jackknife: {j:.4f}, Bootstrap: {b:.4f}")

Доля интервалов для равномерного распределения:
Размер выборки 10: Jackknife: 0.9127, Bootstrap: 0.9082
Размер выборки 100: Jackknife: 0.9453, Bootstrap: 0.9450
Размер выборки 1000: Jackknife: 0.9481, Bootstrap: 0.9473

Доля интервалов для нормального распределения:
Размер выборки 10: Jackknife: 0.9164, Bootstrap: 0.8966
Размер выборки 100: Jackknife: 0.9495, Bootstrap: 0.9481
Размер выборки 1000: Jackknife: 0.9492, Bootstrap: 0.9496
